In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

path = 'https://github.com/oregon-data-science/DSCI101/raw/main/data/'

'imports OK'

## Table review with census data

In [ ]:
## Read in census data

In [ ]:
full = Table.read_table(path + 'nc-est2014-agesex-res.csv')
full

In [ ]:
# Keep only the columns we care about
partial = full.select('SEX', 'AGE', 'POPESTIMATE2010', 'POPESTIMATE2014')
partial

In [ ]:
# Make things easier to read
simple = partial.relabeled(2, '2010').relabeled(3, '2014')
simple

In [ ]:
# Sort by age
simple.sort('AGE')

In [ ]:
# Remove the age totals (the 999 catagory), use are.below
no_999 = simple.where('AGE', are.below(999))
no_999 

In [ ]:
# Remove male and female (keep only combined 0 catagory)
everyone = no_999.where('SEX', 0).drop('SEX')
everyone

In [ ]:
## Males and Females in 2014 ##

In [ ]:
# Let's compare male and female counts per age
males = no_999.where('SEX', 1).drop('SEX')
females = no_999.where('SEX', 2).drop('SEX')

In [ ]:
females.num_rows

In [ ]:
males.num_rows

In [ ]:
## put them back together in a table
## only works because they are the name number of rows!!
pop_2014 = Table().with_columns(
    'Age', males.column('AGE'),  
    'Males', males.column('2014'),
    'Females', females.column('2014')
)
pop_2014

In [ ]:
pop_2014.plot('Age')

In [ ]:
# Calculate the percent female for each age
total = pop_2014.column('Males') + pop_2014.column('Females')
total

In [ ]:
pct_female = (pop_2014.column('Females') / total) * 100
pct_female

In [ ]:
# Round it to 3 so that it's easier to read
pct_female = np.round(pct_female, 2)
pct_female

In [ ]:
# Add female percent to our table
pop_2014 = pop_2014.with_column('Percent female', pct_female)
pop_2014

In [ ]:
pop_2014.plot('Age', 'Percent female')

In [ ]:
# ^^ Look at the y-axis! Trend is not as dramatic as you might think
pop_2014.plot('Age', 'Percent female')
plots.ylim(0, 100);  # Optional for Data 101

## More ways to manipulate rows 

In [ ]:
## Selecting data in a column 

In [ ]:
movies = Table.read_table(path + 'movies_by_year_with_ticket_price.csv')
movies.show()

In [ ]:
## How much each movie made in dollars, use .column so can use array functions 
gross_in_dollars = movies.column('Total Gross') * 1e6
gross_in_dollars

In [ ]:
## How many tickets were sold of each movie? $ each movie made/average price of a ticket
tix_sold = gross_in_dollars / movies.column('Average Ticket Price')
tix_sold

In [ ]:
## Add our new column onto the original table
movies = movies.with_column('Tickets sold', tix_sold)
movies.show(4)

In [ ]:
## a way to make the tickets column more readable
movies = movies.set_format('Tickets sold', NumberFormatter)
movies.show(4)

In [ ]:
## Let's investigate the number of tickets sold in each year using a line plot
movies.plot('Year', 'Tickets sold')

In [ ]:
## New flavors of where, subsetting to a range using are.between "predicate"
movies.where('Year', are.between(2000, 2005))

In [ ]:
## compare that to what we have been previously using, an exact match 
movies.where('Year', 2002)

In [ ]:
## The default predicate that is called "under the hood" 
movies.where('Year', are.equal_to(2002))

In [ ]:
## Another where flavor, matching to a pattern
movies.where('#1 Movie', are.containing('Harry Potter'))

In [ ]:
## Another where flavor, matching to a pattern
movies.where('#1 Movie', are.containing('Lord'))

In [ ]:
## what if we wanted to be systematic about this, imagine there were many rows to count up 

In [ ]:
hp = movies.where('#1 Movie', are.containing('Harry Potter'))
lotr = movies.where('#1 Movie', are.containing('Lord'))

In [ ]:
hp.num_rows, lotr.num_rows

In [ ]:
print("HR top movies=", hp.num_rows, "vs. LOTR top movies=", lotr.num_rows)

In [ ]:
## what other patterns would be interesting to look at??
movies.where('#1 Movie', are.containing(...))

In [ ]:
## Comparing the number of tickets sold in the years HR vs. LOTR were top

In [ ]:
hp.column("Tickets sold")

In [ ]:
lotr.column("Tickets sold")

In [ ]:
## How can we compare the tickets sold? 

In [ ]:
...

## Visualizing numerical data: Scatter plots vs. line plots

In [ ]:
# Actors and their highest grossing movies
actors = Table.read_table(path + 'actors.csv')
actors

In [ ]:
## Make a scatter plot
actors.scatter('Number of Movies', 'Total Gross')

In [ ]:
## What about a line plot (AH!) 
actors.plot('Number of Movies', 'Total Gross')

In [ ]:
## our x variable is not sequential, leads to mayhem, scatter is best 

In [ ]:
## Are there any actors that are one hit wonders? 
actors.scatter('...', '...')

In [ ]:
## to Help decide which columns to use
actors.labels

In [ ]:
## or depending on preference... 
actors.show(5)

In [ ]:
## Who is that crazy point with < 10 movies but a crazy high average gross???? 

In [ ]:
actors.where('Average per Movie', ...)

## Visualizing categorical data: barcharts

In [ ]:
# Highest grossing movies as of 2017
top_movies = Table.read_table(path + 'top_movies_2017.csv')
top_movies

In [ ]:
## subset to the top 10, how? 
top10_adjusted = top_movies.take(...)
top10_adjusted

In [ ]:
# Convert to millions of dollars for readability, round
millions = np.round(top10_adjusted.column('Gross (Adjusted)') / 1e6, 2)
top10_adjusted = top10_adjusted.with_column('Millions', millions)
top10_adjusted

In [ ]:
# A line plot doesn't make sense here: don't do this! AH! 
top10_adjusted.plot('Year', 'Millions')

In [ ]:
## Use a barchar instead
top10_adjusted.barh('Title', 'Millions')

In [ ]:
## When did we start keeping track of movies
min(top_movies.column("Year"))